In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
# Load the dataset with the appropriate encoding
data = pd.read_csv('/content/IMDb Movies India.csv', encoding='latin1')
data

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali
...,...,...,...,...,...,...,...,...,...,...
15504,Zulm Ko Jala Doonga,(1988),NaN,Action,4.6,11,Mahendra Shah,Naseeruddin Shah,Sumeet Saigal,Suparna Anand
15505,Zulmi,(1999),129 min,"Action, Drama",4.5,655,Kuku Kohli,Akshay Kumar,Twinkle Khanna,Aruna Irani
15506,Zulmi Raj,(2005),NaN,Action,NaN,NaN,Kiran Thej,Sangeeta Tiwari,NaN,NaN
15507,Zulmi Shikari,(1988),NaN,Action,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Update the column names as needed
data.rename(columns={'Director': 'director', 'Actor 1': 'actors', 'Genre': 'genre', 'Rating': 'rating'}, inplace=True)

In [ ]:
# Data Preprocessing
# Handle missing values in the 'rating' column
data.dropna(subset=['rating'], inplace=True)

In [ ]:
# Convert 'rating' column to a numerical data type (e.g., float)
data['rating'] = data['rating'].astype(float)
data['rating']


1        7.0
3        4.4
5        4.7
6        7.4
8        5.6
        ... 
15501    5.3
15503    5.8
15504    4.6
15505    4.5
15508    6.2
Name: rating, Length: 7919, dtype: float64

In [ ]:
# Encode categorical features into numerical values
label_encoder = LabelEncoder()
data['director'] = label_encoder.fit_transform(data['director'])
data['director']

1         811
3        1749
5        2005
6        2643
8         174
         ... 
15501     500
15503    2339
15504    1418
15505    1338
15508    1171
Name: director, Length: 7919, dtype: int64

In [ ]:
data['actors'] = label_encoder.fit_transform(data['actors'])
data['actors']

1        1782
3        1589
5         508
6         931
8        2520
         ... 
15501     631
15503     550
15504    1405
15505     145
15508     631
Name: actors, Length: 7919, dtype: int64

In [ ]:
data['genre'] = label_encoder.fit_transform(data['genre'])
data['genre']

1        268
3        207
5        177
6        331
8        367
        ... 
15501     28
15503     28
15504      0
15505     38
15508     38
Name: genre, Length: 7919, dtype: int64

In [ ]:
# Filter out rows with large or invalid 'rating' values (e.g., consider a reasonable range)
data = data[(data['rating'] >= 0) & (data['rating'] <= 10)]
data

,Name,Year,Duration,genre,rating,Votes,director,actors,Actor 2,Actor 3
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,268,7.0,8,811,1782,Vivek Ghamande,Arvind Jangid
3,#Yaaram,(2019),110 min,207,4.4,35,1749,1589,Ishita Raj,Siddhant Kapoor
5,...Aur Pyaar Ho Gaya,(1997),147 min,177,4.7,827,2005,508,Aishwarya Rai Bachchan,Shammi Kapoor
6,...Yahaan,(2005),142 min,331,7.4,"1,086",2643,931,Minissha Lamba,Yashpal Sharma
8,?: A Question Mark,(2012),82 min,367,5.6,326,174,2520,Muntazir Ahmad,Kiran Bhatia
...,...,...,...,...,...,...,...,...,...,...
15501,Zulm Ki Hukumat,(1992),NaN,28,5.3,135,500,631,Moushumi Chatterjee,Govinda
15503,Zulm Ki Zanjeer,(1989),125 min,28,5.8,44,2339,550,Jayamalini,Rajinikanth
15504,Zulm Ko Jala Doonga,(1988),NaN,0,4.6,11,1418,1405,Sumeet Saigal,Suparna Anand
15505,Zulmi,(1999),129 min,38,4.5,655,1338,145,Twinkle Khanna,Aruna Irani


In [ ]:

# Split the data into features (X) and target (y)
X = data[['director', 'actors', 'genre']]
X

,director,actors,genre
1,811,1782,268
3,1749,1589,207
5,2005,508,177
6,2643,931,331
8,174,2520,367
...,...,...,...
15501,500,631,28
15503,2339,550,28
15504,1418,1405,0
15505,1338,145,38


In [ ]:
y = data['rating']
y

1        7.0
3        4.4
5        4.7
6        7.4
8        5.6
        ... 
15501    5.3
15503    5.8
15504    4.6
15505    4.5
15508    6.2
Name: rating, Length: 7919, dtype: float64

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Build and train the regression model (XGBoost in this case)
model = XGBRegressor()
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
# Make predictions on the test data
y_pred = model.predict(X_test)
y_pred

array([4.475114 , 6.6206975, 4.9649763, ..., 4.603895 , 5.8613634,
       6.2016597], dtype=float32)

In [ ]:
# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1.6838765008494592
